<a href="https://colab.research.google.com/github/sdgroeve/ML-course-VIB-2020/blob/master/notebooks/Histone_marks_PyCaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Histone modifications

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

random_seed = 123
np.random.seed(random_seed)

# 1. Reading the data

In [2]:
import pandas as pd

train = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_test.csv")
test_labels = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_test_label.csv")

In [3]:
test["Label"] = test_labels["Label"]

In [4]:
data = pd.concat([train,test])

In [5]:
data.shape

(15485, 502)

In [6]:
data = data.sample(frac=1)

In [7]:
train = data.iloc[0:3000,:]
test = data.iloc[3000:,:]

In [8]:
train

,GeneId,H3K4me3_0,H3K4me1_0,H3K36me3_0,H3K9me3_0,H3K27me3_0,H3K4me3_1,H3K4me1_1,H3K36me3_1,H3K9me3_1,H3K27me3_1,H3K4me3_2,H3K4me1_2,H3K36me3_2,H3K9me3_2,H3K27me3_2,H3K4me3_3,H3K4me1_3,H3K36me3_3,H3K9me3_3,H3K27me3_3,H3K4me3_4,H3K4me1_4,H3K36me3_4,H3K9me3_4,H3K27me3_4,H3K4me3_5,H3K4me1_5,H3K36me3_5,H3K9me3_5,H3K27me3_5,H3K4me3_6,H3K4me1_6,H3K36me3_6,H3K9me3_6,H3K27me3_6,H3K4me3_7,H3K4me1_7,H3K36me3_7,H3K9me3_7,...,H3K4me1_92,H3K36me3_92,H3K9me3_92,H3K27me3_92,H3K4me3_93,H3K4me1_93,H3K36me3_93,H3K9me3_93,H3K27me3_93,H3K4me3_94,H3K4me1_94,H3K36me3_94,H3K9me3_94,H3K27me3_94,H3K4me3_95,H3K4me1_95,H3K36me3_95,H3K9me3_95,H3K27me3_95,H3K4me3_96,H3K4me1_96,H3K36me3_96,H3K9me3_96,H3K27me3_96,H3K4me3_97,H3K4me1_97,H3K36me3_97,H3K9me3_97,H3K27me3_97,H3K4me3_98,H3K4me1_98,H3K36me3_98,H3K9me3_98,H3K27me3_98,H3K4me3_99,H3K4me1_99,H3K36me3_99,H3K9me3_99,H3K27me3_99,Label
433,1180,2,1,1,1,3,2,0,2,1,1,1,1,2,1,1,2,0,2,2,2,0,0,1,0,1,0,2,0,0,1,0,2,0,0,2,0,1,0,1,...,1,1,2,1,3,2,1,2,3,1,3,0,2,4,2,1,2,0,5,2,1,1,2,3,1,2,1,1,5,1,2,1,0,1,1,1,4,0,2,0
6804,13029,4,0,0,0,2,3,2,1,0,0,2,2,0,0,0,0,1,0,1,1,0,1,1,0,0,2,2,1,1,0,3,3,1,0,3,1,2,0,1,...,2,4,0,2,1,2,6,2,4,0,0,3,1,3,2,2,2,3,1,0,4,2,3,1,1,3,2,3,2,1,1,5,3,2,0,1,3,1,3,1
5691,4596,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,3,1,1,0,0,2,2,1,1,2,1,1,1,2,1,7,0,0,...,2,2,2,1,0,0,0,3,2,0,0,0,1,2,0,0,0,2,1,0,1,0,1,0,0,0,1,0,0,0,2,0,1,0,2,0,0,0,0,1
7349,5082,0,0,0,1,1,0,0,2,0,1,0,1,0,0,1,0,2,1,0,1,0,2,2,2,0,0,0,2,2,2,1,1,0,0,2,1,1,0,3,...,3,7,1,0,0,0,5,0,0,0,2,1,1,0,0,2,8,0,0,0,1,8,0,0,0,0,4,2,1,1,0,9,1,3,0,1,6,0,4,1
125,13380,2,1,0,4,4,2,0,0,1,5,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,2,1,0,3,1,3,0,0,2,1,3,1,0,3,...,3,4,5,1,1,1,2,1,0,2,0,0,0,1,1,1,3,1,0,0,1,2,0,0,0,2,2,0,1,1,0,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5139,6610,0,5,2,2,1,0,2,1,5,0,0,2,0,3,0,1,1,0,0,0,2,1,1,1,2,1,1,1,0,0,3,0,2,0,2,1,1,2,2,...,1,6,0,1,0,4,9,4,5,2,2,6,4,3,1,4,6,6,2,1,2,9,3,0,3,1,9,2,1,0,3,7,2,3,1,1,10,0,0,1
4545,3688,2,3,0,2,2,2,2,3,3,3,2,2,1,2,2,8,2,0,1,2,6,0,0,1,4,1,0,1,0,0,1,0,0,0,0,4,0,0,2,...,0,1,1,0,0,0,1,1,0,2,0,2,0,2,2,4,1,0,2,2,2,0,0,0,0,0,0,0,0,1,1,1,0,0,1,3,0,0,1,0
10091,7949,0,1,0,1,0,1,1,1,1,0,0,0,1,1,0,1,0,2,1,0,1,1,0,1,0,1,3,1,1,0,2,0,2,0,0,0,1,2,1,...,6,5,1,0,1,3,2,1,1,1,4,1,2,0,0,3,3,0,3,1,8,2,1,2,0,2,2,1,0,1,2,3,1,2,2,4,0,1,0,1
7541,7702,0,1,4,1,0,0,1,4,0,1,0,0,4,1,0,0,2,7,2,0,0,5,8,3,1,0,2,7,3,2,2,3,8,3,2,4,3,7,3,...,1,0,1,2,4,1,0,1,0,6,3,0,2,1,4,1,0,1,2,5,3,2,1,2,6,2,1,0,2,7,2,1,0,2,4,2,0,3,2,1


In [9]:
train_index_col = train.pop("GeneId")

In [10]:
test_index_col = test.pop("GeneId")

In [12]:
! pip install pycaret

     |████████████████████████████████| 256kB 7.1MB/s 
     |████████████████████████████████| 1.6MB 30.2MB/s 
     |████████████████████████████████| 266kB 44.5MB/s 
     |████████████████████████████████| 174kB 42.0MB/s 
     |████████████████████████████████| 1.8MB 46.1MB/s 
     |████████████████████████████████| 102kB 10.8MB/s 
     |████████████████████████████████| 6.8MB 40.2MB/s 
     |████████████████████████████████| 13.9MB 324kB/s 
     |████████████████████████████████| 276kB 44.2MB/s 
     |████████████████████████████████| 148.9MB 75kB/s 
     |████████████████████████████████| 66.3MB 125kB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 3.1MB 46.7MB/s 
     |████████████████████████████████| 604kB 39.0MB/s 
     |████████████████████████████████| 2.1MB 43.5MB/s 
     |████████████████████████████████| 337kB 46.2MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 

In [13]:
from pycaret.classification import *

In [14]:
#models()

In [15]:
setup(data=data,target="Label",numeric_features=list(data.columns)[:-1], 
      train_size=0.8, silent=True, use_gpu=False)

,Description,Value
0,session_id,6910
1,Target,Label
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(15485, 502)"
5,Missing Values,False
6,Numeric Features,501
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


(3877    0
 5809    0
 2145    0
 7141    0
 3278    0
        ..
 2441    1
 8779    0
 8348    1
 1117    0
 5148    0
 Name: Label, Length: 12388, dtype: int64,
 'Label',
 'lightgbm',
 False,
 False,
        GeneId  H3K4me3_0  H3K4me1_0  ...  H3K36me3_99  H3K9me3_99  H3K27me3_99
 3877   9813.0        0.0        0.0  ...          0.0         0.0          0.0
 5809  11133.0        1.0        2.0  ...          0.0         0.0          1.0
 2145  14934.0        0.0        4.0  ...          4.0         1.0          2.0
 7141   1631.0        2.0        0.0  ...          1.0         1.0          0.0
 3278  14379.0        5.0        0.0  ...          1.0         7.0          3.0
 ...       ...        ...        ...  ...          ...         ...          ...
 2441   8139.0        0.0        1.0  ...          4.0         3.0          1.0
 8779   5047.0        1.0        3.0  ...          1.0         3.0          2.0
 8348   4532.0        1.0        2.0  ...          1.0         1.0          0

In [16]:
from sklearn.metrics import log_loss

add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7ff847b79268>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [17]:
compare_models(cross_validation=False,sort="LogLoss")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
catboost,CatBoost Classifier,0.8547,0.9215,0.8704,0.8419,0.8560,0.7095,0.7099,0.3555,49.81
lightgbm,Light Gradient Boosting Machine,0.8492,0.9158,0.8613,0.8389,0.8500,0.6985,0.6987,0.3688,3.38
lr,Logistic Regression,0.8482,0.9095,0.8535,0.8425,0.8480,0.6965,0.6966,0.3873,12.31
lda,Linear Discriminant Analysis,0.8440,0.9091,0.8424,0.8430,0.8427,0.6881,0.6881,0.3918,1.87
rf,Random Forest Classifier,0.8511,0.9091,0.8704,0.8361,0.8530,0.7024,0.7030,0.4015,7.79
et,Extra Trees Classifier,0.8537,0.9095,0.8763,0.8365,0.8560,0.7076,0.7083,0.4133,5.08
xgboost,Extreme Gradient Boosting,0.8424,0.9086,0.8561,0.8312,0.8435,0.6849,0.6852,0.4493,24.02
ada,Ada Boost Classifier,0.8308,0.8986,0.8301,0.8290,0.8295,0.6616,0.6616,0.6750,8.85
knn,K Neighbors Classifier,0.8005,0.8593,0.7441,0.8355,0.7872,0.6005,0.6041,1.9821,0.97
qda,Quadratic Discriminant Analysis,0.8198,0.8739,0.8275,0.8127,0.8200,0.6397,0.6398,5.1862,1.38


In [18]:
from pycaret.internal.distributions import *

tune_distributions = {
            "num_leaves": IntUniformDistribution(10, 20),
            "learning_rate": UniformDistribution(0.000001, 0.5, log=True),
            "n_estimators": IntUniformDistribution(10, 300),
            "min_split_gain": UniformDistribution(0, 1),
            "reg_alpha": UniformDistribution(0.0000000001, 10, log=True),
            "reg_lambda": UniformDistribution(0.0000000001, 10, log=True),
            "min_data_in_leaf": IntUniformDistribution(10, 10000),
            "feature_fraction": UniformDistribution(0.4, 1),
            "bagging_fraction": UniformDistribution(0.4, 1),
            "bagging_freq": IntUniformDistribution(1, 7),
            "min_child_samples": IntUniformDistribution(5, 100),
        }

In [19]:
lightgbm = create_model("lightgbm",folds=5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.8644,0.9252,0.8698,0.8615,0.8656,0.7288,0.7288,0.3446
1,0.8450,0.9124,0.8424,0.8479,0.8452,0.6901,0.6901,0.3789
2,0.8434,0.9072,0.8505,0.8397,0.8450,0.6868,0.6869,0.3890
3,0.8644,0.9275,0.8730,0.8592,0.8660,0.7288,0.7289,0.3394
4,0.8555,0.9249,0.8859,0.8361,0.8603,0.7110,0.7123,0.3501
5,0.8523,0.9191,0.8521,0.8535,0.8528,0.7046,0.7046,0.3631
6,0.8660,0.9321,0.8714,0.8631,0.8672,0.7320,0.7321,0.3299
7,0.8475,0.9112,0.8427,0.8523,0.8475,0.6949,0.6950,0.3791
8,0.8473,0.9176,0.8617,0.8388,0.8501,0.6946,0.6949,0.3683
9,0.8489,0.9167,0.8698,0.8362,0.8526,0.6978,0.6984,0.3744


In [20]:
from pycaret.internal.utils import (
    param_grid_to_lists,
    get_logger,
    get_class_name,
    np_list_arange,
)

tune_grid = {
            "num_leaves": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200],
            "learning_rate": np_list_arange(0.001, 0.5, 0.001, inclusive=True),
            "n_estimators": np_list_arange(10, 300, 10, inclusive=True),
            "min_split_gain": [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
            "reg_alpha": [
                0.0000001,
                0.000001,
                0.0001,
                0.001,
                0.01,
                0.0005,
                0.005,
                0.05,
                0.1,
                0.15,
                0.2,
                0.3,
                0.4,
                0.5,
                0.7,
                1,
                2,
                3,
                4,
                5,
                10,
            ],
            "reg_lambda": [
                0.0000001,
                0.000001,
                0.0001,
                0.001,
                0.01,
                0.0005,
                0.005,
                0.05,
                0.1,
                0.15,
                0.2,
                0.3,
                0.4,
                0.5,
                0.7,
                1,
                2,
                3,
                4,
                5,
                10,
            ],
            "feature_fraction": np_list_arange(0.4, 1, 0.1, inclusive=True),
            "bagging_fraction": np_list_arange(0.4, 1, 0.1, inclusive=True),
            "bagging_freq": [1, 2, 3, 4, 5, 6, 7],
            "min_child_samples": np_list_arange(5, 100, 5, inclusive=True),
        }

tuned_lightgbm = tune_model(lightgbm, optimize="LogLoss", tuner_verbose=1, n_iter=10,
                            custom_grid=tune_grid)

IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:50:43
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: ignored

# 2. Fitting a decision tree model

The scikit-learn `DecisionTreeClassifier` class computes a decision tree predictive model from a dataset. 

To get all the options for learning you can simply type: 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
help(DecisionTreeClassifier)

You notice that there are many (hyper)parameters to set. These influence the complexity of the model. An important such parameter is the `max_depth` that sets a limit on how deep a decision tree can become. 

Let's create a decision tree model with `max_depth=3`:

In [ ]:
cls = DecisionTreeClassifier(max_depth=3)

This creates a decision tree model with default values for the other hyperparameters:

In [ ]:
cls

Let's create a validation set, fit the model and evaluate.

In [ ]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train,train_labels,
                                                  test_size=.2, random_state=random_seed)

cls.fit(train_X,train_y)

predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)

print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))

predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)

print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,-1]),log_loss(val_y,predictions_val_prob[:,-1])))

The following code plots the fitted decision tree `cls` as a `tree.png` file:

In [ ]:
"""
from sklearn import tree
from io import StringIO
from IPython.display import Image, display
import pydotplus

out = StringIO()
tree.export_graphviz(cls_DT, out_file=out)
graph=pydotplus.graph_from_dot_data(out.getvalue())
graph.write_png("tree.png")
"""

How do other values for for the `max_depth` hyperparameter perform?

In [ ]:
import seaborn as sns

result = []
for md in range(1,10):
  cls = DecisionTreeClassifier(max_depth=md)
  cls.fit(train_X,train_y)
  predictions_train_prob = cls.predict_proba(train_X)
  predictions_val_prob = cls.predict_proba(val_X)
  result.append([md,log_loss(train_y,predictions_train_prob[:,-1]),"train"])
  result.append([md,log_loss(val_y,predictions_val_prob[:,-1]),"val"])

toplot = pd.DataFrame(result,columns=["max_depth","log-loss","set"])
sns.lmplot(x="max_depth",y="log-loss",hue="set",data=toplot,fit_reg=False)

In [ ]:
cls = DecisionTreeClassifier(max_depth=14)

predictions_list = []
for i in range(10):
  train_X, val_X, train_y, val_y = train_test_split(train,train_labels,
                                                    test_size=.2, random_state=i)

  cls.fit(train_X,train_y)
  predictions_val = cls.predict(val_X)
  predictions_val_prob = cls.predict_proba(val_X)
  predictions_list.append(list(predictions_val_prob[:,-1]))
  print("%f %f"%(log_loss(val_y,predictions_val_prob[:,-1]),accuracy_score(val_y,predictions_val)))

In [ ]:
tmp = pd.DataFrame(predictions_list)
predictions_avg = tmp.mean(axis=0)
print("Avg. model: %f"%(log_loss(val_y,predictions_avg)))

In [ ]:

#pd.plotting.scatter_matrix(tmp.transpose())
sns.pairplot(tmp.transpose(),kind="scatter")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

scaler_minmax = preprocessing.MinMaxScaler()
scaler_minmax.fit(train)
train_norm = pd.DataFrame(scaler_minmax.transform(train),columns=train.columns)

#cls = DecisionTreeClassifier(max_depth=4)
cls = LogisticRegression(C=0.5)

predictions_list = []
for i in range(10):
  train_X, val_X, train_y, val_y = train_test_split(train_norm,train_labels,
                                                    test_size=.2, random_state=i)

  cls.fit(train_X,train_y)
  predictions_val = cls.predict(val_X)
  predictions_val_prob = cls.predict_proba(val_X)
  predictions_list.append(list(predictions_val_prob[:,-1]))
  print("%f %f"%(log_loss(val_y,predictions_val_prob[:,-1]),accuracy_score(val_y,predictions_val)))

In [ ]:
tmp = pd.DataFrame(predictions_list)
#pd.plotting.scatter_matrix(tmp.transpose())
sns.pairplot(tmp.transpose(),kind="scatter")

In [ ]:
predictions_avg = tmp.mean(axis=0)
print("Avg. model: %f"%(log_loss(val_y,predictions_avg)))

# 5. Hyperparamters

For our first submission we set the hyperparameter `max_depth=3`. Other values might result in lower log-loss on the testset. 

Since we don't have the testset labels we can only check this on the public leaderboard, which we can/should not do!

So, we need to create our own testset (**not seen during training!**) with known class labels.

Scikit-learn offers many options to do this. One of them is the `train_test_split` function:

In [ ]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(train_norm,train_labels,
                                                  test_size=.2, random_state=random_seed)

#train fold
print(train_X.shape)
print(train_y.shape)
#validation fold
print(val_X.shape)
print(val_y.shape)

Fit a decision tree model with `max_depth=14` default paramters on the `train_X` data set.

In [ ]:
#solution
cls_DT = DecisionTreeClassifier(max_depth=14)
cls_DT.fit(train_X,train_y)

What is the accuracy and log-loss on `train_X`? 

In [ ]:
#solution
predictions = cls_DT.predict(train_X)
print("Accuracy: %f"%(accuracy_score(predictions, train_y)))
predictions = cls_DT.predict_proba(train_X)
print("Log-loss: %f"%(log_loss(train_y,predictions[:,1])))

What is the accuracy and log-loss on `val_X`?

In [ ]:
#solution
predictions = cls_DT.predict(val_X)
print("Accuracy: %f"%(accuracy_score(predictions, val_y)))
predictions = cls_DT.predict_proba(val_X)
print("Log-loss: %f"%(log_loss(val_y,predictions[:,1])))

What do you see?


The following code evaluates different values for this hyperparameter.

In [ ]:
for maxdepth in range(1,20,1):
    cls = DecisionTreeClassifier(max_depth=maxdepth)
    cls.fit(train_X,train_y)
    predictions_train = cls.predict(train_X)
    predictions_val = cls.predict(val_X)
    predictions_train_prob = cls.predict_proba(train_X)[:,1]
    predictions_val_prob = cls.predict_proba(val_X)[:,1]
    print("%i (%f) %f (%f) %f"%(maxdepth,
                                accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y),
                                log_loss(train_y,predictions_train_prob),log_loss(val_y,predictions_val_prob)))

What do you see?

So, we have split the data into a train- and validationset. We can of course split the data in many different ways (different random seeds) resulting in different train- and validationsets.

Let's try 5 different random seeds:

In [ ]:
for run in range(5):
  train_X, val_X, train_y, val_y = train_test_split(train_norm,train_labels,
                                                  test_size=.8, random_state=run)
  min_m = 100
  best = None
  for maxdepth in range(1,20,1):
      cls = DecisionTreeClassifier(max_depth=maxdepth)
      cls.fit(train_X,train_y)
      predictions_val_prob = cls.predict_proba(val_X)[:,1]
      m = log_loss(val_y,predictions_val_prob)
      if m < min_m:
        min_m = m
        best = maxdepth
  print("%i %f"%(best,min_m))

What do you see?

The solution is to run several train-validations splits and average the performance.

One popular method is cross-validation that uses each datapoint once as a testpoint.

It works as follows:
<br/>
<br/>
<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png"/>
<br/>
<br/>

It is easy to run this in Scikit-learn:



In [ ]:
from sklearn.model_selection import cross_val_predict

for maxdepth in range(1,10,1):
    cls = DecisionTreeClassifier(max_depth=maxdepth)
    predictions = cross_val_predict(cls,train_norm,train_labels,
                                    cv=10,
                                    method="predict_proba")
    print("%i %f"%(maxdepth,log_loss(train_labels,predictions[:,1])))

We can do this in two lines of code with the `GridSearchCV` module:


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth':range(1,10)
    }

GSCV = GridSearchCV(cls_DT, params,
                    cv=10,
                    scoring="neg_log_loss",
                    verbose=1).fit(train_norm,train_labels)

print(GSCV.best_estimator_)
print(GSCV.best_score_)

Play with the hyperparameters in a Template notebook and make some Kaggle submissions.

# 5. Ensemble learning: bagging

We have seen that bias and variance play an important role in Machine Learning. 

Let's first see what bagging can do for our dataset. 

In [ ]:
from sklearn.ensemble import BaggingClassifier

cls = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=4),random_state=random_seed)
                                                            
cls.fit(train_X,train_y)
predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)
print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))
predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)
print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,1]),log_loss(val_y,predictions_val_prob[:,1])))

With the `RandomForestClassifier` the variance of the decision tree is reduced also by selecting features for decision tree contruction at random. Let's see how far we get with default hyperparameter values.   

In [ ]:
from sklearn.ensemble import RandomForestClassifier

cls = RandomForestClassifier(random_state=random_seed)

cls.fit(train_X,train_y)
predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)
print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))
predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)
print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,1]),log_loss(val_y,predictions_val_prob[:,1])))

# 6. Ensemble learning: boosting

How about the `GradientBoostingClassifier`?

In [ ]:
#solution
from sklearn.ensemble import GradientBoostingClassifier

cls = GradientBoostingClassifier(random_state=random_seed,
                                    max_depth=10)
cls.fit(train_X,train_y)
predictions_train = cls.predict(train_X)
predictions_val = cls.predict(val_X)
print("Accuracy: (%f) %f"%(accuracy_score(predictions_train, train_y),accuracy_score(predictions_val, val_y)))
predictions_train_prob = cls.predict_proba(train_X)
predictions_val_prob = cls.predict_proba(val_X)
print("Log-loss: (%f) %f"%(log_loss(train_y,predictions_train_prob[:,1]),log_loss(val_y,predictions_val_prob[:,1])))